In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import numpy as np
import os

In [15]:
driver = webdriver.Chrome()
driver.get("https://archive.vogue.com/login")

In [ ]:
start_date = pd.to_datetime("1920-01-01")
end_date = pd.to_datetime("1930-01-01")

term = ["mini dress"]

def make_term_search(driver, term, start_date, end_date):
    url = f"https://archive.vogue.com/search?QueryTerm={term}&startDate={start_date}&endDate={end_date}"
    driver.get(url)

def get_count(driver):
    count_elements = driver.find_elements(By.CLASS_NAME, "count")
    if not count_elements:
        return None
    count_text = count_elements[0].text
    count = int(count_text.split("(")[1].split(")")[0])
    return count


increment = pd.DateOffset(years=1)
results = []
current_date = start_date

os.makedirs("vogue_resource_dump", exist_ok=True)

while current_date < end_date:
    next_date = current_date + increment
    print(f"Searching from {current_date} to {next_date}")

    make_term_search(driver, term, current_date.strftime("%Y-%m-%d"), next_date.strftime("%Y-%m-%d"))

    count = get_count(driver)
    print(f"{count} results found")

    with open(f"vogue_resource_dump/{term}_{current_date.strftime('%Y-%m-%d')}_{next_date.strftime('%Y-%m-%d')}.html", "w") as f:
        f.write(driver.page_source)

    results.append({
        "Term": term,
        "Start Date": current_date,
        "End Date": next_date,
        "Result Count": count
    })

    current_date = next_date

    # Sleep to avoid detection, ensuring non-negative sleep length
    sleep_duration = max(0, np.random.normal(1, 0.5))
    time.sleep(sleep_duration)

driver.quit()

financialcrash = pd.DataFrame(results)
print("Search Results Summary:")
print(financialcrash)